In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from src.utilities import *
from src.features.activity_feature import extract_activity_features

In [6]:
raw_activity = pd.read_csv(os.path.join(raw_path, 'activity.csv'))
raw_activity=raw_activity.rename(columns = {'count':'counts'})
raw_activity.head()

,hcp,specialty,region,brand,month,channel,counts
0,39972,Internal medicine / pneumology,region_61,brand_1,2020-05,video,1
1,64026,General practicioner,region_168,brand_2,2020-05,phone,1
2,27892,Internal medicine,region_26,brand_2,2020-06,phone,1
3,33500,General practicioner,region_26,brand_2,2020-06,phone,1
4,61239,General practicioner,region_26,brand_2,2020-06,phone,1


In [30]:
def add_month(month):
    '''Given a string with month format of YYYY-MM add one month'''

    if (int(month[5:])<12):
        if (int(month[5:]) + 1 >= 10):
            return month[:5] + str(int(month[5:]) + 1)
        return month[:5] + '0' + str(int(month[5:]) + 1)
    return str(int(month[:4]) + 1) + '-01'

In [9]:
activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                          'specialty']).counts.sum().unstack().reset_index().fillna(0)
activity_features_specialty.isna().sum()

specialty
month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
dtype: int64

In [10]:
#activities to different specialty per month
activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                          'specialty']).counts.sum().unstack().reset_index().fillna(0)
activity_features_specialty['act_total_1mo'] = activity_features_specialty.sum(axis = 1)
activity_features_specialty['act_GP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['General practicioner'].sum())
activity_features_specialty['act_IM_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine'].sum())
activity_features_specialty['act_IMP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine / pneumology'].sum())
activity_features_specialty['act_IMGP_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Internal medicine and general practicioner'].sum())
activity_features_specialty['act_PE_1mo'] = activity_features_specialty.month.map(
    activity_features_specialty.groupby('month')['Pediatrician'].sum())

In [12]:
activity_features_specialty

specialty
month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
act_total_1mo                                 0
act_GP_1mo                                    0
act_IM_1mo                                    0
act_IMP_1mo                                   0
act_IMGP_1mo                                  0
act_PE_1mo                                    0
dtype: int64

In [15]:
#activities via different channel per month
activity_features_channel = raw_activity.groupby(['month', 'region', 'brand', 
                                          'channel']).counts.sum().unstack().reset_index().fillna(0)
activity_features_channel['act_f2f_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['f2f'].sum())
activity_features_channel['act_other_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['other'].sum())
activity_features_channel['act_phone_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['phone'].sum())
activity_features_channel['act_video_1mo'] = activity_features_channel.month.map(
    activity_features_channel.groupby('month')['video'].sum())
activity_features_channel.isna().sum()

channel
month            0
region           0
brand            0
f2f              0
other            0
phone            0
video            0
act_f2f_1mo      0
act_other_1mo    0
act_phone_1mo    0
act_video_1mo    0
dtype: int64

In [17]:
#activity to different specialty per month per region
GP_monthly = activity_features_specialty.groupby(['month','region'])['General practicioner'].sum().reset_index()
GP_monthly.columns = ['month', 'region', 'act_GP_region_1mo']
IM_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine'].sum().reset_index()
IM_monthly.columns = ['month', 'region', 'act_IM_region_1mo']
IMP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine / pneumology'].sum().reset_index()
IMP_monthly.columns = ['month', 'region', 'act_IMP_region_1mo']
IMGP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine and general practicioner'].sum().reset_index()
IMGP_monthly.columns = ['month', 'region', 'act_IMGP_region_1mo']
PE_monthly = activity_features_specialty.groupby(['month','region'])['Pediatrician'].sum().reset_index()
PE_monthly.columns = ['month', 'region', 'act_PE_region_1mo']

activity_features_specialty = activity_features_specialty.merge(GP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IM_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IMP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(IMGP_monthly, how = 'outer', on = ['month', 'region'])
activity_features_specialty = activity_features_specialty.merge(PE_monthly, how = 'outer', on = ['month', 'region'])

activity_features_specialty.isna().sum()

month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
act_total_1mo                                 0
act_GP_1mo                                    0
act_IM_1mo                                    0
act_IMP_1mo                                   0
act_IMGP_1mo                                  0
act_PE_1mo                                    0
act_GP_region_1mo_x                           0
act_IM_region_1mo_x                           0
act_IMP_region_1mo_x                          0
act_IMGP_region_1mo_x                         0
act_PE_region_1mo_x                           0
act_GP_region_1mo_y                           0
act_IM_region_1mo_y                     

In [18]:
#activity via different channel per month per region
f2f_monthly = activity_features_channel.groupby(['month','region'])['f2f'].sum().reset_index()
f2f_monthly.columns = ['month', 'region', 'act_f2f_region_1mo']
phone_monthly = activity_features_channel.groupby(['month','region'])['phone'].sum().reset_index()
phone_monthly.columns = ['month', 'region', 'act_phone_region_1mo']
video_monthly = activity_features_channel.groupby(['month','region'])['video'].sum().reset_index()
video_monthly.columns = ['month', 'region', 'act_video_region_1mo']
other_monthly = activity_features_channel.groupby(['month','region'])['other'].sum().reset_index()
other_monthly.columns = ['month', 'region', 'act_other_region_1mo']

activity_features_channel = activity_features_channel.merge(f2f_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(phone_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(video_monthly, how = 'outer', on = ['month', 'region'])
activity_features_channel = activity_features_channel.merge(other_monthly, how = 'outer', on = ['month', 'region'])

activity_features_channel.isna().sum()

month                   0
region                  0
brand                   0
f2f                     0
other                   0
phone                   0
video                   0
act_f2f_1mo             0
act_other_1mo           0
act_phone_1mo           0
act_video_1mo           0
act_f2f_region_1mo      0
act_phone_region_1mo    0
act_video_region_1mo    0
act_other_region_1mo    0
dtype: int64

In [30]:
# groupby = master.value_counts(['month', 'brand']).reset_index(name = 'monthly brand activity')
# master['monthly brand activity'] = groupby['monthly brand activity']

In [19]:
#activity to different specialty per month per brand
GP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['General practicioner'].sum().reset_index()
GP_monthly_brand.columns = ['month', 'brand', 'act_GP_brand_1mo']
IM_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine'].sum().reset_index()
IM_monthly_brand.columns = ['month', 'brand', 'act_IM_brand_1mo']
IMP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine / pneumology'].sum().reset_index()
IMP_monthly_brand.columns = ['month', 'brand', 'act_IMP_brand_1mo']
IMGP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine and general practicioner'].sum().reset_index()
IMGP_monthly_brand.columns = ['month', 'brand', 'act_IMGP_brand_1mo']
PE_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Pediatrician'].sum().reset_index()
PE_monthly_brand.columns = ['month', 'brand', 'act_PE_brand_1mo']

activity_features_specialty = activity_features_specialty.merge(GP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IM_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IMP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(IMGP_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_specialty = activity_features_specialty.merge(PE_monthly_brand, how = 'outer', on = ['month', 'brand'])

activity_features_specialty.isna().sum()

month                                         0
region                                        0
brand                                         0
General practicioner                          0
Internal medicine                             0
Internal medicine / pneumology                0
Internal medicine and general practicioner    0
Pediatrician                                  0
act_total_1mo                                 0
act_GP_1mo                                    0
act_IM_1mo                                    0
act_IMP_1mo                                   0
act_IMGP_1mo                                  0
act_PE_1mo                                    0
act_GP_region_1mo_x                           0
act_IM_region_1mo_x                           0
act_IMP_region_1mo_x                          0
act_IMGP_region_1mo_x                         0
act_PE_region_1mo_x                           0
act_GP_region_1mo_y                           0
act_IM_region_1mo_y                     

In [20]:
#activity via different channel per month per brand
f2f_monthly_brand = activity_features_channel.groupby(['month','brand'])['f2f'].sum().reset_index()
f2f_monthly_brand.columns = ['month', 'brand', 'act_f2f_brand_1mo']
phone_monthly_brand = activity_features_channel.groupby(['month','brand'])['phone'].sum().reset_index()
phone_monthly_brand.columns = ['month', 'brand', 'act_phone_brand_1mo']
video_monthly_brand = activity_features_channel.groupby(['month','brand'])['video'].sum().reset_index()
video_monthly_brand.columns = ['month', 'brand', 'act_video_brand_1mo']
other_monthly_brand = activity_features_channel.groupby(['month','brand'])['other'].sum().reset_index()
other_monthly_brand.columns = ['month', 'brand', 'act_other_brand_1mo']

activity_features_channel = activity_features_channel.merge(f2f_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(phone_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(video_monthly_brand, how = 'outer', on = ['month', 'brand'])
activity_features_channel = activity_features_channel.merge(other_monthly_brand, how = 'outer', on = ['month', 'brand'])

activity_features_channel.isna().sum()

month                   0
region                  0
brand                   0
f2f                     0
other                   0
phone                   0
video                   0
act_f2f_1mo             0
act_other_1mo           0
act_phone_1mo           0
act_video_1mo           0
act_f2f_region_1mo      0
act_phone_region_1mo    0
act_video_region_1mo    0
act_other_region_1mo    0
act_f2f_brand_1mo       0
act_phone_brand_1mo     0
act_video_brand_1mo     0
act_other_brand_1mo     0
dtype: int64

In [22]:
activity_features = activity_features_channel.merge(activity_features_specialty, 
                                                    how = 'outer', on = ['month', 'region','brand'])



In [23]:
activity_features.isna().sum()

month                                         0
region                                        0
brand                                         0
f2f                                           0
other                                         0
phone                                         0
video                                         0
act_f2f_1mo                                   0
act_other_1mo                                 0
act_phone_1mo                                 0
act_video_1mo                                 0
act_f2f_region_1mo                            0
act_phone_region_1mo                          0
act_video_region_1mo                          0
act_other_region_1mo                          0
act_f2f_brand_1mo                             0
act_phone_brand_1mo                           0
act_video_brand_1mo                           0
act_other_brand_1mo                           0
General practicioner                          0
Internal medicine                       

In [24]:
activity_features=activity_features.rename(columns = {'f2f':'act_f2f_region_brand_1mo', 'other':'act_other_region_brand_1mo',
                                                     'phone':'act_phone_region_brand_1mo','video':'act_video_region_brand_1mo',
                                                     'General practicioner':'act_GP_region_brand_1mo', 'Internal medicine':'act_IM_region_brand_1mo',
                                                     'Internal medicine / pneumology':'act_IMP_region_brand_1mo',
                                                     'Internal medicine and general practicioner':'act_IMPG_region_brand_1mo',
                                                     'Pediatrician':'act_PE_region_brand_1mo'})

In [25]:
activity_features.isna().sum()

month                         0
region                        0
brand                         0
act_f2f_region_brand_1mo      0
act_other_region_brand_1mo    0
act_phone_region_brand_1mo    0
act_video_region_brand_1mo    0
act_f2f_1mo                   0
act_other_1mo                 0
act_phone_1mo                 0
act_video_1mo                 0
act_f2f_region_1mo            0
act_phone_region_1mo          0
act_video_region_1mo          0
act_other_region_1mo          0
act_f2f_brand_1mo             0
act_phone_brand_1mo           0
act_video_brand_1mo           0
act_other_brand_1mo           0
act_GP_region_brand_1mo       0
act_IM_region_brand_1mo       0
act_IMP_region_brand_1mo      0
act_IMPG_region_brand_1mo     0
act_PE_region_brand_1mo       0
act_total_1mo                 0
act_GP_1mo                    0
act_IM_1mo                    0
act_IMP_1mo                   0
act_IMGP_1mo                  0
act_PE_1mo                    0
act_GP_region_1mo_x           0
act_IM_r

In [52]:
def extract_activity_features(raw_activity: pd.DataFrame) -> pd.DataFrame:

    raw_activity.sort_values('month', inplace = True)
    
    #activities to different specialty per month
    activity_features_specialty = raw_activity.groupby(['month', 'region', 'brand', 
                                              'specialty']).counts.sum().unstack().reset_index().fillna(0)
    activity_features_specialty['act_total_1mo'] = activity_features_specialty.sum(axis = 1)
    activity_features_specialty['act_GP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['General practicioner'].sum())
    activity_features_specialty['act_IM_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine'].sum())
    activity_features_specialty['act_IMP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine / pneumology'].sum())
    activity_features_specialty['act_IMGP_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Internal medicine and general practicioner'].sum())
    activity_features_specialty['act_PE_1mo'] = activity_features_specialty.month.map(
        activity_features_specialty.groupby('month')['Pediatrician'].sum())
    
    #activities via different channel per month
    activity_features_channel = raw_activity.groupby(['month', 'region', 'brand', 
                                              'channel']).counts.sum().unstack().reset_index().fillna(0)
    activity_features_channel['act_f2f_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['f2f'].sum())
    activity_features_channel['act_other_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['other'].sum())
    activity_features_channel['act_phone_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['phone'].sum())
    activity_features_channel['act_video_1mo'] = activity_features_channel.month.map(
        activity_features_channel.groupby('month')['video'].sum())
    activity_features_channel

    #activity to different specialty per month per region
    GP_monthly = activity_features_specialty.groupby(['month','region'])['General practicioner'].sum().reset_index()
    GP_monthly.columns = ['month', 'region', 'act_GP_region_1mo']
    IM_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine'].sum().reset_index()
    IM_monthly.columns = ['month', 'region', 'act_IM_region_1mo']
    IMP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine / pneumology'].sum().reset_index()
    IMP_monthly.columns = ['month', 'region', 'act_IMP_region_1mo']
    IMGP_monthly = activity_features_specialty.groupby(['month','region'])['Internal medicine and general practicioner'].sum().reset_index()
    IMGP_monthly.columns = ['month', 'region', 'act_IMGP_region_1mo']
    PE_monthly = activity_features_specialty.groupby(['month','region'])['Pediatrician'].sum().reset_index()
    PE_monthly.columns = ['month', 'region', 'act_PE_region_1mo']

    activity_features_specialty = activity_features_specialty.merge(GP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IM_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IMP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(IMGP_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_specialty = activity_features_specialty.merge(PE_monthly, how = 'outer', on = ['month', 'region'])

    #activity to different specialty per month per brand
    GP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['General practicioner'].sum().reset_index()
    GP_monthly_brand.columns = ['month', 'brand', 'act_GP_brand_1mo']
    IM_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine'].sum().reset_index()
    IM_monthly_brand.columns = ['month', 'brand', 'act_IM_brand_1mo']
    IMP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine / pneumology'].sum().reset_index()
    IMP_monthly_brand.columns = ['month', 'brand', 'act_IMP_brand_1mo']
    IMGP_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Internal medicine and general practicioner'].sum().reset_index()
    IMGP_monthly_brand.columns = ['month', 'brand', 'act_IMGP_brand_1mo']
    PE_monthly_brand = activity_features_specialty.groupby(['month','brand'])['Pediatrician'].sum().reset_index()
    PE_monthly_brand.columns = ['month', 'brand', 'act_PE_brand_1mo']

    activity_features_specialty = activity_features_specialty.merge(GP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IM_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IMP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(IMGP_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_specialty = activity_features_specialty.merge(PE_monthly_brand, how = 'outer', on = ['month', 'brand'])

    #activity via different channel per month per region
    f2f_monthly = activity_features_channel.groupby(['month','region'])['f2f'].sum().reset_index()
    f2f_monthly.columns = ['month', 'region', 'act_f2f_region_1mo']
    phone_monthly = activity_features_channel.groupby(['month','region'])['phone'].sum().reset_index()
    phone_monthly.columns = ['month', 'region', 'act_phone_region_1mo']
    video_monthly = activity_features_channel.groupby(['month','region'])['video'].sum().reset_index()
    video_monthly.columns = ['month', 'region', 'act_video_region_1mo']
    other_monthly = activity_features_channel.groupby(['month','region'])['other'].sum().reset_index()
    other_monthly.columns = ['month', 'region', 'act_other_region_1mo']

    activity_features_channel = activity_features_channel.merge(f2f_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(phone_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(video_monthly, how = 'outer', on = ['month', 'region'])
    activity_features_channel = activity_features_channel.merge(other_monthly, how = 'outer', on = ['month', 'region'])



    #activity via different channel per month per brand
    f2f_monthly_brand = activity_features_channel.groupby(['month','brand'])['f2f'].sum().reset_index()
    f2f_monthly_brand.columns = ['month', 'brand', 'act_f2f_brand_1mo']
    phone_monthly_brand = activity_features_channel.groupby(['month','brand'])['phone'].sum().reset_index()
    phone_monthly_brand.columns = ['month', 'brand', 'act_phone_brand_1mo']
    video_monthly_brand = activity_features_channel.groupby(['month','brand'])['video'].sum().reset_index()
    video_monthly_brand.columns = ['month', 'brand', 'act_video_brand_1mo']
    other_monthly_brand = activity_features_channel.groupby(['month','brand'])['other'].sum().reset_index()
    other_monthly_brand.columns = ['month', 'brand', 'act_other_brand_1mo']

    activity_features_channel = activity_features_channel.merge(f2f_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(phone_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(video_monthly_brand, how = 'outer', on = ['month', 'brand'])
    activity_features_channel = activity_features_channel.merge(other_monthly_brand, how = 'outer', on = ['month', 'brand'])

    
    activity_features = activity_features_channel.merge(activity_features_specialty, 
                                                    how = 'outer', on = ['month', 'region','brand'])


    activity_features=activity_features.rename(columns = {'f2f':'act_f2f_region_brand_1mo', 'other':'act_other_region_brand_1mo',
                                                     'phone':'act_phone_region_brand_1mo','video':'act_video_region_brand_1mo',
                                                     'General practicioner':'act_GP_region_brand_1mo', 'Internal medicine':'act_IM_region_brand_1mo',
                                                     'Internal medicine / pneumology':'act_IMP_region_brand_1mo',
                                                     'Internal medicine and general practicioner':'act_IMPG_region_brand_1mo',
                                                     'Pediatrician':'act_PE_region_brand_1mo'})
    
    activity_features.month = activity_features.month.apply(lambda x: add_month(x))

    # Add rolling aggregations of the previously created features
    cols = [c for c in activity_features.columns if c not in ['month', 'region', 'brand']]
    
    for c in cols:
        activity_features[c[:-3] + '2mo'] = activity_features.groupby(['region', 'brand'])[c].shift()
        activity_features[c[:-3] + '3mo'] = activity_features.groupby(['region', 'brand'])[c].shift(2)
        activity_features[c[:-3] + '4mo'] = activity_features.groupby(['region', 'brand'])[c].shift(3)
        
        activity_features[ c[:-3] + 'trend_2mo'] = activity_features[c[:-3] + '2mo'] / (activity_features[c[:-3] + '1mo']+1)
        activity_features[c[:-3] + 'trend_3mo'] = activity_features[c[:-3] + '3mo'] / (activity_features[c[:-3] + '1mo']+1)
        activity_features[ c[:-3] + 'trend_4mo'] = activity_features[c[:-3] + '4mo'] / (activity_features[c[:-3] + '1mo']+1)

    months = pd.DataFrame({'month': raw_activity.month.unique()})
    regions = pd.DataFrame({'region': raw_activity.region.unique()})
    brands = pd.DataFrame({'brand': ['brand_1', 'brand_2']})

    months['dummy_col'] = 0
    regions['dummy_col'] = 0
    brands['dummy_col'] = 0

    activity_features_full = months.merge(regions, how = 'outer', on = 'dummy_col')
    activity_features_full = activity_features_full.merge(brands, how = 'outer', on = 'dummy_col')
    activity_features_full.drop(columns = 'dummy_col', inplace = True)
    activity_features_full.sort_values('month', inplace = True)
    activity_features = activity_features.merge(activity_features_full,how='outer',on=['month','region','brand'])
    activity_features = activity_features.fillna(0)
    return activity_features


In [58]:
#checking 
raw_activity[(raw_activity.month=='2020-10')&(raw_activity.region=='region_70')&
             (raw_activity.channel=='f2f')&(raw_activity.brand=='brand_1')].counts.sum()

22

In [57]:
#checking
temp = extract_activity_features(raw_activity)
temp[(temp.brand=='brand_1')&(temp.region=='region_70')][['month', 'brand', 'region', 
                                                           'act_f2f_region_brand_1mo','act_f2f_region_brand_2mo', 'act_f2f_region_brand_3mo']]

,month,brand,region,act_f2f_region_brand_1mo,act_f2f_region_brand_2mo,act_f2f_region_brand_3mo
365,2020-09,brand_1,region_70,25.0,0.0,0.0
752,2020-10,brand_1,region_70,11.0,25.0,0.0
1145,2020-11,brand_1,region_70,22.0,11.0,25.0
1518,2020-12,brand_1,region_70,0.0,22.0,11.0
1804,2021-01,brand_1,region_70,0.0,0.0,22.0
2294,2021-03,brand_1,region_70,0.0,0.0,0.0
2539,2021-04,brand_1,region_70,5.0,0.0,0.0
2772,2021-05,brand_1,region_70,0.0,5.0,0.0
3000,2021-06,brand_1,region_70,2.0,0.0,5.0
3301,2021-07,brand_1,region_70,9.0,2.0,0.0


In [50]:
result = extract_activity_features(raw_activity)
result.groupby(['month']).size()

month
2020-05    402
2020-06    402
2020-07    402
2020-08    402
2020-09    402
2020-10    402
2020-11    402
2020-12    402
2021-01    402
2021-02    402
2021-03    402
2021-04    402
2021-05    402
2021-06    402
2021-07    402
2021-08    402
2021-09    402
2021-10    307
dtype: int64

In [53]:
extract_activity_features(raw_activity).groupby('month').apply(lambda x: x.isna().mean())

,month,region,brand,act_f2f_region_brand_1mo,act_other_region_brand_1mo,act_phone_region_brand_1mo,act_video_region_brand_1mo,act_f2f_1mo,act_other_1mo,act_phone_1mo,act_video_1mo,act_f2f_region_1mo,act_phone_region_1mo,act_video_region_1mo,act_other_region_1mo,act_f2f_brand_1mo,act_phone_brand_1mo,act_video_brand_1mo,act_other_brand_1mo,act_GP_region_brand_1mo,act_IM_region_brand_1mo,act_IMP_region_brand_1mo,act_IMPG_region_brand_1mo,act_PE_region_brand_1mo,act_total_1mo,act_GP_1mo,act_IM_1mo,act_IMP_1mo,act_IMGP_1mo,act_PE_1mo,act_GP_region_1mo,act_IM_region_1mo,act_IMP_region_1mo,act_IMGP_region_1mo,act_PE_region_1mo,act_GP_brand_1mo,act_IM_brand_1mo,act_IMP_brand_1mo,act_IMGP_brand_1mo,act_PE_brand_1mo,act_f2f_region_brand_2mo,act_f2f_region_brand_3mo,act_f2f_region_brand_4mo,act_f2f_region_brand_trend_2mo,act_f2f_region_brand_trend_3mo,act_f2f_region_brand_trend_4mo,act_other_region_brand_2mo,act_other_region_brand_3mo,act_other_region_brand_4mo,act_other_region_brand_trend_2mo,act_other_region_brand_trend_3mo,act_other_region_brand_trend_4mo,act_phone_region_brand_2mo,act_phone_region_brand_3mo,act_phone_region_brand_4mo,act_phone_region_brand_trend_2mo,act_phone_region_brand_trend_3mo,act_phone_region_brand_trend_4mo,act_video_region_brand_2mo,act_video_region_brand_3mo,act_video_region_brand_4mo,act_video_region_brand_trend_2mo,act_video_region_brand_trend_3mo,act_video_region_brand_trend_4mo,act_f2f_2mo,act_f2f_3mo,act_f2f_4mo,act_f2f_trend_2mo,act_f2f_trend_3mo,act_f2f_trend_4mo,act_other_2mo,act_other_3mo,act_other_4mo,act_other_trend_2mo,act_other_trend_3mo,act_other_trend_4mo,act_phone_2mo,act_phone_3mo,act_phone_4mo,act_phone_trend_2mo,act_phone_trend_3mo,act_phone_trend_4mo,act_video_2mo,act_video_3mo,act_video_4mo,act_video_trend_2mo,act_video_trend_3mo,act_video_trend_4mo,act_f2f_region_2mo,act_f2f_region_3mo,act_f2f_region_4mo,act_f2f_region_trend_2mo,act_f2f_region_trend_3mo,act_f2f_region_trend_4mo,act_phone_region_2mo,act_phone_region_3mo,act_phone_region_4mo,act_phone_region_trend_2mo,act_phone_region_trend_3mo,act_phone_region_trend_4mo,act_video_region_2mo,act_video_region_3mo,act_video_region_4mo,act_video_region_trend_2mo,act_video_region_trend_3mo,act_video_region_trend_4mo,act_other_region_2mo,act_other_region_3mo,act_other_region_4mo,act_other_region_trend_2mo,act_other_region_trend_3mo,act_other_region_trend_4mo,act_f2f_brand_2mo,act_f2f_brand_3mo,act_f2f_brand_4mo,act_f2f_brand_trend_2mo,act_f2f_brand_trend_3mo,act_f2f_brand_trend_4mo,act_phone_brand_2mo,act_phone_brand_3mo,act_phone_brand_4mo,act_phone_brand_trend_2mo,act_phone_brand_trend_3mo,act_phone_brand_trend_4mo,act_video_brand_2mo,act_video_brand_3mo,act_video_brand_4mo,act_video_brand_trend_2mo,act_video_brand_trend_3mo,act_video_brand_trend_4mo,act_other_brand_2mo,act_other_brand_3mo,act_other_brand_4mo,act_other_brand_trend_2mo,act_other_brand_trend_3mo,act_other_brand_trend_4mo,act_GP_region_brand_2mo,act_GP_region_brand_3mo,act_GP_region_brand_4mo,act_GP_region_brand_trend_2mo,act_GP_region_brand_trend_3mo,act_GP_region_brand_trend_4mo,act_IM_region_brand_2mo,act_IM_region_brand_3mo,act_IM_region_brand_4mo,act_IM_region_brand_trend_2mo,act_IM_region_brand_trend_3mo,act_IM_region_brand_trend_4mo,act_IMP_region_brand_2mo,act_IMP_region_brand_3mo,act_IMP_region_brand_4mo,act_IMP_region_brand_trend_2mo,act_IMP_region_brand_trend_3mo,act_IMP_region_brand_trend_4mo,act_IMPG_region_brand_2mo,act_IMPG_region_brand_3mo,act_IMPG_region_brand_4mo,act_IMPG_region_brand_trend_2mo,act_IMPG_region_brand_trend_3mo,act_IMPG_region_brand_trend_4mo,act_PE_region_brand_2mo,act_PE_region_brand_3mo,act_PE_region_brand_4mo,act_PE_region_brand_trend_2mo,act_PE_region_brand_trend_3mo,act_PE_region_brand_trend_4mo,act_total_2mo,act_total_3mo,act_total_4mo,act_total_trend_2mo,act_total_trend_3mo,act_total_trend_4mo,act_GP_2mo,act_GP_3mo,act_GP_4mo,act_GP_trend_2mo,act_GP_trend_3mo,act_GP_trend_4mo,act_IM_2mo,act_IM_3mo,act_IM_4mo,act_IM_trend_2mo,act_IM_trend

In [59]:
raw_activity['brand_region'] = raw_activity['brand'] + raw_activity['region']
raw_activity.groupby(['month']).brand_region.nunique()

month
2020-05      2
2020-06      7
2020-07    201
2020-08    375
2020-09    393
2020-10    384
2020-11    320
2020-12    252
2021-01    246
2021-02    242
2021-03    257
2021-04    199
2021-05    275
2021-06    335
2021-07    340
2021-08    302
2021-09    307
Name: brand_region, dtype: int64

In [142]:
raw_rtes = pd.read_csv(os.path.join(raw_path, 'rtes.csv'))

In [73]:
raw_rtes.head()

,hcp,specialty,region,brand,email_type,content_id,no. openings,no. clicks,time_sent,time_last_opened,time_last_clicked,month
0,58555,General practicioner,region_0,brand_1,general,a3D6f0000019jMvEAI,1,0,2021-03-02 08:42:00,2021-03-02 08:45:00,0,2021-03
1,819,General practicioner,region_1,brand_2,product_related,a3D6f0000019gA2EAI,0,0,2020-07-16 11:58:00,0,0,2020-07
2,819,General practicioner,region_1,brand_2,general,a3D6f0000019jSnEAI,1,0,2020-08-17 14:47:00,2020-08-17 23:16:00,0,2020-08
3,819,General practicioner,region_1,brand_1,product_related,a3D6f000000MtZ3EAK,0,0,2020-09-03 16:08:00,0,0,2020-09
4,819,General practicioner,region_1,brand_1,general,a3D6f000000N0nhEAC,1,0,2020-10-16 13:35:00,2020-10-16 22:39:00,0,2020-10


In [143]:
raw_rtes=raw_rtes.fillna(0)

In [144]:
raw_rtes['time_sent'] = pd.to_datetime(raw_rtes.time_sent, format='%Y-%m-%d')
raw_rtes['month'] = raw_rtes['time_sent'].dt.strftime('%Y-%m')

In [114]:
raw_rtes.specialty.value_counts()

General practicioner                          21474
Internal medicine                             11814
Internal medicine / pneumology                 7860
Internal medicine and general practicioner      336
Pediatrician                                    184
Name: specialty, dtype: int64

In [145]:
# 1 - total counts at a month-region-brand-level
rtes_total = raw_rtes.groupby(['month', 'region', 
                           'brand']).agg({'hcp': 'count', 
                                          'no. openings': 'sum',
                                          'no. clicks': 'sum'
                                         }).reset_index()
rtes_total.columns = ['month', 'region', 'brand', 'rtes_totalemails_1mo', 'rtes_totalopenings_1mo', 'rtes_totalclicks_1mo']
rtes_total.head()

,month,region,brand,rtes_totalemails_1mo,rtes_totalopenings_1mo,rtes_totalclicks_1mo
0,2020-07,region_1,brand_2,107,74,3
1,2020-07,region_10,brand_2,115,90,13
2,2020-07,region_100,brand_2,2,49,0
3,2020-07,region_101,brand_2,46,40,0
4,2020-07,region_102,brand_2,19,134,3


In [185]:
# 2 - Breaking down the counts by type of email
rtes_feature_type = raw_rtes.groupby(['month', 'region', 'brand',
                                              'email_type']).size().unstack().reset_index().fillna(0)
rtes_feature_type.columns = ['month', 'region', 'brand', 'rtes_typegeneralcount_brand_1mo', 'rtes_typeprodcount_brand_1mo', 'rtes_typeevent_brand_1mo']
rtes_feature_type.head()

,month,region,brand,rtes_typegeneralcount_brand_1mo,rtes_typeprodcount_brand_1mo,rtes_typeevent_brand_1mo
0,2020-07,region_1,brand_2,0.0,107.0,0.0
1,2020-07,region_10,brand_2,56.0,59.0,0.0
2,2020-07,region_100,brand_2,1.0,1.0,0.0
3,2020-07,region_101,brand_2,0.0,46.0,0.0
4,2020-07,region_102,brand_2,0.0,19.0,0.0


In [186]:
rtes_feature_type

,month,region,brand,rtes_typegeneralcount_brand_1mo,rtes_typeprodcount_brand_1mo,rtes_typeevent_brand_1mo
0,2020-07,region_1,brand_2,0.0,107.0,0.0
1,2020-07,region_10,brand_2,56.0,59.0,0.0
2,2020-07,region_100,brand_2,1.0,1.0,0.0
3,2020-07,region_101,brand_2,0.0,46.0,0.0
4,2020-07,region_102,brand_2,0.0,19.0,0.0
...,...,...,...,...,...,...
1967,2021-09,region_93,brand_1,3.0,0.0,0.0
1968,2021-09,region_94,brand_1,17.0,0.0,0.0
1969,2021-09,region_96,brand_1,6.0,0.0,0.0
1970,2021-09,region_97,brand_1,4.0,0.0,0.0


In [147]:
# 3 - Type of emails per month per region
rtes_feature_type_month_region = raw_rtes.groupby(['month', 'region','email_type']).size().unstack().reset_index().fillna(0)
rtes_feature_type_month_region.columns = ['month', 'region', 'rtes_typegeneralcount_1mo', 'rtes_typeprodcount_1mo', 'rtes_typeevent_1mo']
rtes_feature_type_month_region.head()

,month,region,rtes_typegeneralcount_1mo,rtes_typeprodcount_1mo,rtes_typeevent_1mo
0,2020-07,region_1,0.0,107.0,0.0
1,2020-07,region_10,56.0,59.0,0.0
2,2020-07,region_100,1.0,1.0,0.0
3,2020-07,region_101,0.0,46.0,0.0
4,2020-07,region_102,0.0,19.0,0.0


In [164]:
# 4 - Types of emails per month per specialty
raw_rtes['type_specialty'] = raw_rtes['email_type']+raw_rtes['specialty']
rtes_feature_type_specialty = raw_rtes.groupby(['month', 'region', 'brand',
                                              'type_specialty']).size().unstack().reset_index().fillna(0)
rtes_feature_type_specialty.columns = ['month','region','brand','generalGP_region_brand_1mo','generalIM_region_brand_1mo','generalIMP_region_brand_1mo','generalIMGP_region_brand_1mo','generalPE_region_brand_1mo',
                                      'productGP_region_brand_1mo','productIM_region_brand_1mo','productIMP_region_brand_1mo','productIMGP_region_brand_1mo','productPE_region_brand_1mo','eventGP_region_brand_1mo','eventIM_region_brand_1mo']
rtes_feature_type_specialty.head()

,month,region,brand,generalGP_region_brand_1mo,generalIM_region_brand_1mo,generalIMP_region_brand_1mo,generalIMGP_region_brand_1mo,generalPE_region_brand_1mo,productGP_region_brand_1mo,productIM_region_brand_1mo,productIMP_region_brand_1mo,productIMGP_region_brand_1mo,productPE_region_brand_1mo,eventGP_region_brand_1mo,eventIM_region_brand_1mo
0,2020-07,region_1,brand_2,0.0,0.0,0.0,0.0,0.0,76.0,28.0,2.0,1.0,0.0,0.0,0.0
1,2020-07,region_10,brand_2,38.0,18.0,0.0,0.0,0.0,38.0,20.0,1.0,0.0,0.0,0.0,0.0
2,2020-07,region_100,brand_2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-07,region_101,brand_2,0.0,0.0,0.0,0.0,0.0,33.0,12.0,1.0,0.0,0.0,0.0,0.0
4,2020-07,region_102,brand_2,0.0,0.0,0.0,0.0,0.0,3.0,6.0,10.0,0.0,0.0,0.0,0.0


In [165]:
rtes_feature_type_specialty_month = rtes_feature_type_specialty.groupby(['month'])['generalGP_region_brand_1mo','generalIM_region_brand_1mo','generalIMP_region_brand_1mo','generalIMGP_region_brand_1mo','generalPE_region_brand_1mo',
                                      'productGP_region_brand_1mo','productIM_region_brand_1mo','productIMP_region_brand_1mo','productIMGP_region_brand_1mo','productPE_region_brand_1mo','eventGP_region_brand_1mo','eventIM_region_brand_1mo'].sum().reset_index()
rtes_feature_type_specialty_month.columns = ['month','generalGP_1mo','generalIM_1mo','generalIMP_1mo','generalIMGP_1mo','generalPE_1mo',
                                      'productGP_1mo','productIM_1mo','productIMP_1mo','productIMGP_1mo','productPE_1mo','eventGP_1mo','eventIM_1mo']
rtes_feature_type_specialty_month.head()



,month,generalGP_1mo,generalIM_1mo,generalIMP_1mo,generalIMGP_1mo,generalPE_1mo,productGP_1mo,productIM_1mo,productIMP_1mo,productIMGP_1mo,productPE_1mo,eventGP_1mo,eventIM_1mo
0,2020-07,466.0,256.0,86.0,3.0,1.0,3143.0,1533.0,567.0,42.0,17.0,36.0,25.0
1,2020-08,2722.0,1426.0,504.0,52.0,8.0,2219.0,1130.0,487.0,47.0,4.0,0.0,0.0
2,2020-09,688.0,315.0,59.0,7.0,1.0,2653.0,1425.0,280.0,39.0,0.0,0.0,0.0
3,2020-10,134.0,47.0,12.0,2.0,0.0,759.0,423.0,298.0,10.0,1.0,0.0,0.0
4,2020-11,221.0,94.0,17.0,3.0,2.0,1382.0,798.0,645.0,28.0,9.0,0.0,0.0


In [149]:
#raw_rtes[(raw_rtes.specialty=='Internal medicine / pneumology')&(raw_rtes.email_type=='regional_events')].count()

hcp                  0
specialty            0
region               0
brand                0
email_type           0
content_id           0
no. openings         0
no. clicks           0
time_sent            0
time_last_opened     0
time_last_clicked    0
month                0
dtype: int64

In [168]:
# 5 - Number of opening per month per region
rtes_feature_nopen_month_region = raw_rtes.groupby(['month', 'region']).agg({'no. openings': 'sum'}).reset_index()
rtes_feature_nopen_month_region.columns = ['month', 'region', 'rtes_nopening_region_1mo']
rtes_feature_nopen_month_region.head()

,month,region,rtes_nopening_region_1mo
0,2020-07,region_1,74
1,2020-07,region_10,90
2,2020-07,region_100,49
3,2020-07,region_101,40
4,2020-07,region_102,134


In [169]:
# 6 - Number of opening per region
rtes_feature_nopen_region = raw_rtes.groupby(['region']).agg({'no. openings': 'sum'}).reset_index()
rtes_feature_nopen_region.columns = ['region','rtes_nopening_region']
rtes_feature_nopen_region.head()

,region,rtes_nopening_region
0,region_0,1
1,region_1,382
2,region_10,275
3,region_100,132
4,region_101,41


In [181]:
# 7 - Number of opening per month per region per specialty
rtes_feature_nopen_specialty_region_month = raw_rtes.groupby(['month', 'region','specialty']).agg({'no. openings': 'sum'}).unstack().reset_index().fillna(0)
rtes_feature_nopen_specialty_region_month.columns = ['month','region','rtes_nopen_GP_region_1mo','rtes_nopen_IM_region_1mo',
                                                   'rtes_nopen_IMP_region_1mo','rtes_nopen_IMGP_region_1mo','rtes_nopen_PE_region_1mo']


In [182]:
rtes_feature_nopen_specialty_region_month

,month,region,rtes_nopen_GP_region_1mo,rtes_nopen_IM_region_1mo,rtes_nopen_IMP_region_1mo,rtes_nopen_IMGP_region_1mo,rtes_nopen_PE_region_1mo
0,2020-07,region_1,55.0,15.0,4.0,0.0,0.0
1,2020-07,region_10,44.0,44.0,2.0,0.0,0.0
2,2020-07,region_100,32.0,0.0,17.0,0.0,0.0
3,2020-07,region_101,20.0,20.0,0.0,0.0,0.0
4,2020-07,region_102,0.0,128.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...
1845,2021-09,region_93,0.0,0.0,10.0,0.0,0.0
1846,2021-09,region_94,0.0,0.0,19.0,0.0,0.0
1847,2021-09,region_96,0.0,0.0,2.0,0.0,0.0
1848,2021-09,region_97,0.0,0.0,6.0,0.0,0.0


In [187]:
final_rtes_features = rtes_total.copy()
final_rtes_features = final_rtes_features.merge(rtes_feature_type, 
                                                how = 'left', on = ['month', 'region', 'brand'])
final_rtes_features = final_rtes_features.merge(rtes_feature_type_month_region, how = 'left', 
                                               on = ['month', 'region'])
final_rtes_features = final_rtes_features.merge(rtes_feature_type_specialty, how= 'left',
                                               on = ['month', 'region', 'brand'])
final_rtes_features = final_rtes_features.merge(rtes_feature_nopen_month_region, how = 'left',
                                               on = ['month', 'region'])
final_rtes_features = final_rtes_features.merge(rtes_feature_nopen_specialty_region_month, how= 'left',
                                               on = ['month', 'region'])

In [190]:
final_rtes_features

,month,region,brand,rtes_totalemails_1mo,rtes_totalopenings_1mo,rtes_totalclicks_1mo,rtes_typegeneralcount_brand_1mo,rtes_typeprodcount_brand_1mo,rtes_typeevent_brand_1mo,rtes_typegeneralcount_1mo,rtes_typeprodcount_1mo,rtes_typeevent_1mo,generalGP_region_brand_1mo,generalIM_region_brand_1mo,generalIMP_region_brand_1mo,generalIMGP_region_brand_1mo,generalPE_region_brand_1mo,productGP_region_brand_1mo,productIM_region_brand_1mo,productIMP_region_brand_1mo,productIMGP_region_brand_1mo,productPE_region_brand_1mo,eventGP_region_brand_1mo,eventIM_region_brand_1mo,rtes_nopening_region_1mo,rtes_nopen_GP_region_1mo,rtes_nopen_IM_region_1mo,rtes_nopen_IMP_region_1mo,rtes_nopen_IMGP_region_1mo,rtes_nopen_PE_region_1mo
0,2020-07,region_1,brand_2,107,74,3,0.0,107.0,0.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,28.0,2.0,1.0,0.0,0.0,0.0,74,55.0,15.0,4.0,0.0,0.0
1,2020-07,region_10,brand_2,115,90,13,56.0,59.0,0.0,56.0,59.0,0.0,38.0,18.0,0.0,0.0,0.0,38.0,20.0,1.0,0.0,0.0,0.0,0.0,90,44.0,44.0,2.0,0.0,0.0
2,2020-07,region_100,brand_2,2,49,0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,49,32.0,0.0,17.0,0.0,0.0
3,2020-07,region_101,brand_2,46,40,0,0.0,46.0,0.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,12.0,1.0,0.0,0.0,0.0,0.0,40,20.0,20.0,0.0,0.0,0.0
4,2020-07,region_102,brand_2,19,134,3,0.0,19.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,10.0,0.0,0.0,0.0,0.0,134,0.0,128.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967,2021-09,region_93,brand_1,3,10,0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.0,0.0,10.0,0.0,0.0
1968,2021-09,region_94,brand_1,17,19,3,17.0,0.0,0.0,17.0,0.0,0.0,0.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,0.0,0.0,19.0,0.0,0.0
1969,2021-09,region_96,brand_1,6,2,0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,2.0,0.0,0.0
1970,2021-09,region_97,brand_1,4,6,1,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,0.0,0.0,6.0,0.0,0.0


In [212]:
def extract_rtes_features(raw_rtes: pd.DataFrame) -> pd.DataFrame:
    raw_rtes=raw_rtes.fillna(0)
    raw_rtes['time_sent'] = pd.to_datetime(raw_rtes.time_sent, format='%Y-%m-%d')
    raw_rtes['month'] = raw_rtes['time_sent'].dt.strftime('%Y-%m')
    # 1 - total counts at a month-region-brand-level
    rtes_total = raw_rtes.groupby(['month', 'region', 
                               'brand']).agg({'hcp': 'count', 
                                              'no. openings': 'sum',
                                              'no. clicks': 'sum'
                                             }).reset_index()
    rtes_total.columns = ['month', 'region', 'brand', 'rtes_totalemails_1mo', 'rtes_totalopenings_1mo', 'rtes_totalclicks_1mo']
    

    # 2 - Breaking down the counts by type of email
    rtes_feature_type = raw_rtes.groupby(['month', 'region', 'brand',
                                                  'email_type']).size().unstack().reset_index().fillna(0)
    rtes_feature_type.columns = ['month', 'region', 'brand', 'rtes_typegeneralcount_brand_1mo', 'rtes_typeprodcount_brand_1mo', 'rtes_typeevent_brand_1mo']

    # 3 - Type of emails per month per region
    rtes_feature_type_month_region = raw_rtes.groupby(['month', 'region','email_type']).size().unstack().reset_index().fillna(0)
    rtes_feature_type_month_region.columns = ['month', 'region', 'rtes_typegeneralcount_1mo', 'rtes_typeprodcount_1mo', 'rtes_typeevent_1mo']

    
    # 4 - Types of emails per month per specialty
    raw_rtes['type_specialty'] = raw_rtes['email_type']+raw_rtes['specialty']
    rtes_feature_type_specialty = raw_rtes.groupby(['month', 'region', 'brand',
                                                  'type_specialty']).size().unstack().reset_index().fillna(0)
    rtes_feature_type_specialty.columns = ['month','region','brand','generalGP_region_brand_1mo','generalIM_region_brand_1mo','generalIMP_region_brand_1mo','generalIMGP_region_brand_1mo','generalPE_region_brand_1mo',
                                          'productGP_region_brand_1mo','productIM_region_brand_1mo','productIMP_region_brand_1mo','productIMGP_region_brand_1mo','productPE_region_brand_1mo','eventGP_region_brand_1mo','eventIM_region_brand_1mo']
    rtes_feature_type_specialty_month = rtes_feature_type_specialty.groupby(['month'])['generalGP_region_brand_1mo','generalIM_region_brand_1mo','generalIMP_region_brand_1mo','generalIMGP_region_brand_1mo','generalPE_region_brand_1mo',
                                      'productGP_region_brand_1mo','productIM_region_brand_1mo','productIMP_region_brand_1mo','productIMGP_region_brand_1mo','productPE_region_brand_1mo','eventGP_region_brand_1mo','eventIM_region_brand_1mo'].sum().reset_index()
    rtes_feature_type_specialty_month.columns = ['month','generalGP_1mo','generalIM_1mo','generalIMP_1mo','generalIMGP_1mo','generalPE_1mo',
                                          'productGP_1mo','productIM_1mo','productIMP_1mo','productIMGP_1mo','productPE_1mo','eventGP_1mo','eventIM_1mo']

    
    # 5 - Number of opening per month per region
    rtes_feature_nopen_month_region = raw_rtes.groupby(['month', 'region']).agg({'no. openings': 'sum'}).reset_index()
    rtes_feature_nopen_month_region.columns = ['month', 'region', 'rtes_nopening_region_1mo']

    # 6 - Number of opening per region
    rtes_feature_nopen_region = raw_rtes.groupby(['region']).agg({'no. openings': 'sum'}).reset_index()
    rtes_feature_nopen_region.columns = ['region','rtes_nopening_region']

    
    # 7 - Number of opening per month per region per specialty
    rtes_feature_nopen_specialty_region_month = raw_rtes.groupby(['month', 'region','specialty']).agg({'no. openings': 'sum'}).unstack().reset_index().fillna(0)
    rtes_feature_nopen_specialty_region_month.columns = ['month','region','rtes_nopen_GP_region_1mo','rtes_nopen_IM_region_1mo',
                                                       'rtes_nopen_IMP_region_1mo','rtes_nopen_IMGP_region_1mo','rtes_nopen_PE_region_1mo']
    final_rtes_features = rtes_total.copy()
    final_rtes_features = final_rtes_features.merge(rtes_feature_type, 
                                                    how = 'left', on = ['month', 'region', 'brand'])
    final_rtes_features = final_rtes_features.merge(rtes_feature_type_month_region, how = 'left', 
                                                   on = ['month', 'region'])
    final_rtes_features = final_rtes_features.merge(rtes_feature_type_specialty, how= 'left',
                                                   on = ['month', 'region', 'brand'])
    final_rtes_features = final_rtes_features.merge(rtes_feature_nopen_month_region, how = 'left',
                                                   on = ['month', 'region'])
    final_rtes_features = final_rtes_features.merge(rtes_feature_nopen_specialty_region_month, how= 'left',
                                                   on = ['month', 'region'])
    final_rtes_features.month = final_rtes_features.month.apply(lambda x: add_month(x))

    # Add rolling aggregations of the previously created features
    cols = [c for c in final_rtes_features.columns if c not in ['month', 'region', 'brand']]
    
    for c in cols:
        final_rtes_features[c[:-3] + '2mo'] = final_rtes_features.groupby(['region', 'brand'])[c].shift()
        final_rtes_features[c[:-3] + '3mo'] = final_rtes_features.groupby(['region', 'brand'])[c].shift(2)
        final_rtes_features[c[:-3] + '4mo'] = final_rtes_features.groupby(['region', 'brand'])[c].shift(3)
        
        final_rtes_features[ c[:-3] + 'trend_2mo'] = final_rtes_features[c[:-3] + '2mo'] / (final_rtes_features[c[:-3] + '1mo']+1)
        final_rtes_features[c[:-3] + 'trend_3mo'] = final_rtes_features[c[:-3] + '3mo'] / (final_rtes_features[c[:-3] + '1mo']+1)
        final_rtes_features[ c[:-3] + 'trend_4mo'] = final_rtes_features[c[:-3] + '4mo'] / (final_rtes_features[c[:-3] + '1mo']+1)

    months = pd.DataFrame({'month': raw_rtes.month.unique()})
    regions = pd.DataFrame({'region': raw_rtes.region.unique()})
    brands = pd.DataFrame({'brand': ['brand_1', 'brand_2']})

    months['dummy_col'] = 0
    regions['dummy_col'] = 0
    brands['dummy_col'] = 0

    final_rtes_features_full = months.merge(regions, how = 'outer', on = 'dummy_col')
    final_rtes_features_full= final_rtes_features_full.merge(brands, how = 'outer', on = 'dummy_col')
    final_rtes_features_full.drop(columns = 'dummy_col', inplace = True)
    final_rtes_features_full.sort_values('month', inplace = True)
    final_rtes_features = final_rtes_features.merge(final_rtes_features_full,how='outer',on=['month','region','brand'])
    final_rtes_features = final_rtes_features.fillna(0)
    return final_rtes_features


In [213]:
temp_rtes = extract_rtes_features(raw_rtes) 

In [241]:
temp_rtes

,month,region,brand,rtes_totalemails_1mo,rtes_totalopenings_1mo,rtes_totalclicks_1mo,rtes_typegeneralcount_brand_1mo,rtes_typeprodcount_brand_1mo,rtes_typeevent_brand_1mo,rtes_typegeneralcount_1mo,rtes_typeprodcount_1mo,rtes_typeevent_1mo,generalGP_region_brand_1mo,generalIM_region_brand_1mo,generalIMP_region_brand_1mo,generalIMGP_region_brand_1mo,generalPE_region_brand_1mo,productGP_region_brand_1mo,productIM_region_brand_1mo,productIMP_region_brand_1mo,productIMGP_region_brand_1mo,productPE_region_brand_1mo,eventGP_region_brand_1mo,eventIM_region_brand_1mo,rtes_nopening_region_1mo,rtes_nopen_GP_region_1mo,rtes_nopen_IM_region_1mo,rtes_nopen_IMP_region_1mo,rtes_nopen_IMGP_region_1mo,rtes_nopen_PE_region_1mo,rtes_totalemails_2mo,rtes_totalemails_3mo,rtes_totalemails_4mo,rtes_totalemails_trend_2mo,rtes_totalemails_trend_3mo,rtes_totalemails_trend_4mo,rtes_totalopenings_2mo,rtes_totalopenings_3mo,rtes_totalopenings_4mo,rtes_totalopenings_trend_2mo,rtes_totalopenings_trend_3mo,rtes_totalopenings_trend_4mo,rtes_totalclicks_2mo,rtes_totalclicks_3mo,rtes_totalclicks_4mo,rtes_totalclicks_trend_2mo,rtes_totalclicks_trend_3mo,rtes_totalclicks_trend_4mo,rtes_typegeneralcount_brand_2mo,rtes_typegeneralcount_brand_3mo,rtes_typegeneralcount_brand_4mo,rtes_typegeneralcount_brand_trend_2mo,rtes_typegeneralcount_brand_trend_3mo,rtes_typegeneralcount_brand_trend_4mo,rtes_typeprodcount_brand_2mo,rtes_typeprodcount_brand_3mo,rtes_typeprodcount_brand_4mo,rtes_typeprodcount_brand_trend_2mo,rtes_typeprodcount_brand_trend_3mo,rtes_typeprodcount_brand_trend_4mo,rtes_typeevent_brand_2mo,rtes_typeevent_brand_3mo,rtes_typeevent_brand_4mo,rtes_typeevent_brand_trend_2mo,rtes_typeevent_brand_trend_3mo,rtes_typeevent_brand_trend_4mo,rtes_typegeneralcount_2mo,rtes_typegeneralcount_3mo,rtes_typegeneralcount_4mo,rtes_typegeneralcount_trend_2mo,rtes_typegeneralcount_trend_3mo,rtes_typegeneralcount_trend_4mo,rtes_typeprodcount_2mo,rtes_typeprodcount_3mo,rtes_typeprodcount_4mo,rtes_typeprodcount_trend_2mo,rtes_typeprodcount_trend_3mo,rtes_typeprodcount_trend_4mo,rtes_typeevent_2mo,rtes_typeevent_3mo,rtes_typeevent_4mo,rtes_typeevent_trend_2mo,rtes_typeevent_trend_3mo,rtes_typeevent_trend_4mo,generalGP_region_brand_2mo,generalGP_region_brand_3mo,generalGP_region_brand_4mo,generalGP_region_brand_trend_2mo,generalGP_region_brand_trend_3mo,generalGP_region_brand_trend_4mo,generalIM_region_brand_2mo,generalIM_region_brand_3mo,generalIM_region_brand_4mo,generalIM_region_brand_trend_2mo,generalIM_region_brand_trend_3mo,generalIM_region_brand_trend_4mo,generalIMP_region_brand_2mo,generalIMP_region_brand_3mo,generalIMP_region_brand_4mo,generalIMP_region_brand_trend_2mo,generalIMP_region_brand_trend_3mo,generalIMP_region_brand_trend_4mo,generalIMGP_region_brand_2mo,generalIMGP_region_brand_3mo,generalIMGP_region_brand_4mo,generalIMGP_region_brand_trend_2mo,generalIMGP_region_brand_trend_3mo,generalIMGP_region_brand_trend_4mo,generalPE_region_brand_2mo,generalPE_region_brand_3mo,generalPE_region_brand_4mo,generalPE_region_brand_trend_2mo,generalPE_region_brand_trend_3mo,generalPE_region_brand_trend_4mo,productGP_region_brand_2mo,productGP_region_brand_3mo,productGP_region_brand_4mo,productGP_region_brand_trend_2mo,productGP_region_brand_trend_3mo,productGP_region_brand_trend_4mo,productIM_region_brand_2mo,productIM_region_brand_3mo,productIM_region_brand_4mo,productIM_region_brand_trend_2mo,productIM_region_brand_trend_3mo,productIM_region_brand_trend_4mo,productIMP_region_brand_2mo,productIMP_region_brand_3mo,productIMP_region_brand_4mo,productIMP_region_brand_trend_2mo,productIMP_region_brand_trend_3mo,productIMP_region_brand_trend_4mo,productIMGP_region_brand_2mo,productIMGP_region_brand_3mo,productIMGP_region_brand_4mo,productIMGP_region_brand_trend_2mo,productIMGP_region_brand_trend_3mo,productIMGP_region_brand_trend_4mo,productPE_region_brand_2mo,productPE_region_brand_3mo,productPE_region_brand_4mo,productPE_region_brand_trend_2mo,productPE_region_brand_trend_3mo,pro

In [217]:
temp_rtes.isna().sum()

month                             0
region                            0
brand                             0
rtes_totalemails_1mo              0
rtes_totalopenings_1mo            0
                                 ..
rtes_nopen_PE_region_4mo          0
rtes_nopen_PE_region_trend_2mo    0
rtes_nopen_PE_region_trend_3mo    0
rtes_nopen_PE_region_trend_4mo    0
brand_region                      0
Length: 193, dtype: int64

In [216]:
temp_rtes['brand_region'] = temp_rtes['brand'] + temp_rtes['region']
temp_rtes.groupby(['month']).brand_region.nunique()

month
2020-07    402
2020-08    402
2020-09    402
2020-10    402
2020-11    402
2020-12    402
2021-01    402
2021-02    402
2021-03    402
2021-04    402
2021-05    402
2021-06    402
2021-07    402
2021-08    402
2021-09    402
2021-10     94
Name: brand_region, dtype: int64

In [240]:
#checking 
raw_rtes[(raw_rtes.month =='2021-06')&(raw_rtes.region=='region_70')&(raw_rtes.brand=='brand_1')]['no. openings'].sum()

12

In [220]:
#checking

temp_rtes[(temp_rtes.brand=='brand_1')&(temp_rtes.region=='region_70')][['month', 'brand', 'region', 
                                                           'rtes_totalemails_1mo',
                                                                         'rtes_totalopenings_1mo']]


,month,brand,region,rtes_totalemails_1mo,rtes_totalopenings_1mo
377,2020-09,brand_1,region_70,48.0,31.0
542,2020-10,brand_1,region_70,44.0,29.0
673,2020-11,brand_1,region_70,4.0,2.0
822,2020-12,brand_1,region_70,5.0,16.0
960,2021-01,brand_1,region_70,8.0,4.0
1106,2021-02,brand_1,region_70,11.0,14.0
1242,2021-03,brand_1,region_70,28.0,12.0
1375,2021-04,brand_1,region_70,11.0,32.0
1525,2021-05,brand_1,region_70,11.0,9.0
1656,2021-06,brand_1,region_70,17.0,6.0
